# Zawaansowane widoki część 2

## Agenda
* Rozbudowujemy nasz projekt o nowe widoki
* Formularze
* Widoki generyczne
* Refaktoryzacja projektu

### Rozbudowujemy nasz projekt o nowe widoki
Dowiedzieliśmy się jak działają widoki, jakie rodzaje widoków posiadamy, była to jednak sucha teoria. Czas na praktykę! Wcześniej stworzyliśmy model **Article**, zatem możemy zrobić z naszego projektu bloga. Strony, które powinniśmy utworzyć podczas tworzenia takiego bloga to:
* Strona z listą artykułów
* Strona konkretnego artykułu
* Strona tworzenia nowego artykułu

Więc do dzieła! ;)

Przejdźmy do tworzenia widoków czyli do pliku **views.py**

1. Stwórzmy najpierw widok który będzie wyświetlał listę wszystkich artykułów

```python
from django.shortcuts import render
from my_app.models import Article

...

def articles(request):
    articles = Article.objects.all().order_by("-published_date")
    return render(request, "articles.html", {
        "articles": articles
    }) 
```

Wyciągamy **queryset** z listą wszystkich artykułów posortowanych od najwcześniejszego.
Następnie renderujemy plik HTML przekazując mu w kontekście zmienną **articles** która jest listą artykułów.

Stwórzmy plik **articles.html** jednak zanim utwórzmy templatkę bazową **base.html**.
W folderze **templates** utwórzmy plik **base.html**:
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>
        {% block title %}
        {% endblock %}
    </title>
</head>
<body>
    {% for message in messages %}
        <div class="alert alert-success">
            {{ message }}
        </div>
    {% endfor %}
    
    {% block content %}
    {% endblock %}
</body>
</html>
```

W folderze **templates** utwórzmy plik **articles.html**

```html

{% extends 'base.html' %}
{% block title %}
    Lista artykułów
{% endblock %}

{% block content %}
    <h1>Lista artykułów</h1>
    <hr>
    {% for article in articles %}
        
        <a href="{% url 'article_detail' id=article.id %}"><h5>{{ article.title }}</h5></a>
        <p>Author: <span>{{ article.author.username }}</span></p>
        <p> 
            Likes: <span style="font-weight: bold;">{{ article.likes }}</span> 
            Dislikes: <span style="font-weight: bold;">{{ article.dislikes }}</span>
        </p>
        <br>
        <br>
    {% endfor %}
{% endblock %}

```
Dzięki temu, że dziedziczmy z `base.html` nie musimy za każdym razem tworzyć tagu head oraz body, interesuje nas tylko zawartość która się zmienia, czyli to co jest wewnątrz **{% block content %}**.
Iterujmy po wszystkich artykułach i wyświetlamy podstawowe informacje o każdym artykule jak tytuł, autor czy liczba like'ów. To co jest nowego to tag **url** który służy do tworzenia linku do widoku konkrentengo artykułu. To co wpisujemy jako pierwszy argument to alias naszego URL'a czyli to co się znajduje w argumencie **name**, drugi argument to dodatkowe parametry takie jak np. id

Ostatecznie zarejestrujmy nasz widok pod adresem **/articles/**

```python
from django.contrib import admin
from django.urls import path

from my_app import views 

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", views.home_view, name="home"),
    path("articles/", views.articles, name="articles"), # <-- rejestracja nowego widoku
]
```

2. Widok konkretnego artykułu

```python
from django.shortcuts import render, get_object_or_404
from my_app.models import Article


def article_detail(request, id):
    article = get_object_or_404(Article, pk=id)
    return render(request, "article_detail.html", {
        "article": article
    })
```

pobieramy instance artykułu z modelu **Article** używając funkcji **get_object_or_404** która wyciągnie z bazy obiekt o podanym id, jeżeli nie znajdzie takiego - zwróci błąd 404.
Renderujemy html przekazując obiekt artykułu

Stwórzmy plik **article_detail.html** wewnątrz folderu **templates**

```html
{% extends 'base.html' %}

{% block title %}
    {{ article.title }}
{% endblock %}

{% block content %}
    <h1>{{ article.title|title }}</h1>
    <p>Author: <span style="font-weight: bold;">{{ article.author.username }}</span></p>
    <hr>
    <p>{{ article.description }}</p>
    <hr>
    <p>Likes: {{ article.likes }}</p>
    <p>Dislikes: {{ article.dislikes }}</p>
{% endblock %}
```

oraz zarejestrujmy ten widok w pliku **urls.py**

```python
from django.contrib import admin
from django.urls import path

from my_app import views 

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", views.home_view, name="home"),
    path("articles/", views.articles, name="articles"),
    path("articles/<int:id>/", views.article_detail, name="article_detail"),
]
```

3. Tworzenie nowego artykułu

Jest to trochę trudniejsza logika, wcześniej używaliśmy tylko metody HTTP GET. Tym razem użyjemy 2 metod, GET do wyświetlania formularza z odpowiednimi polami, oraz POST do przyjmowania pól z formularza i tworzenia nowego obiektu modelu `Article`. Użyjmy zatem widoku klasowego:

```python
from django.views import View
from django.contrib.auth.models import User
from django.shortcuts import render
from django.contrib. import messages
from django.utils import timezone

from my_app.models import Article

...


class ArticleCreate(View):

    def get(self, request):
        users = User.objects.all()
        return render(request, "article_create.html", {"users": users})

    def post(self, request):
        title = request.POST.get("title")
        description = request.POST.get("description")
        author = request.POST.get("author")
        author_object = User.objects.get(username=author)

        try:
            article = Article.objects.create(
                title=title,
                description=description,
                author=author_object,
            )
            messages.success(request, "Article with title {} has been created!".format(title))
        except Exception as err:
            messages.success(request, "There was an error: {}".format(err))
        
        return redirect("articles")
```

Omówmy najpierw metodę get() Pobieramy listę wszystkich użytkowników, po to żeby w formularzu wyświetlić dropdown z użytkownikami.

W metodzie post() najpierw odbieramy wszystkie pola z formularza. Następnie tworzymy obiekt artykułu, tą akcję owrapowałem w blok **try-except** żeby złapać ewentualne błędy, używamy **Django messages framework** do wyświetlania odpowiednich wiadomości, więcej o tym [tutaj](https://docs.djangoproject.com/en/2.2/ref/contrib/messages/). Na końcu używając funkcji **redirect()** przechodzimy do widoku listy artykułów, zamiast podawać absolutną ścieżkę np. `https://mysite.com/articles/` możemy się odwołać do adresu używając jego aliasu czyli tego co wpisaliśmy w parametrze **name**

Stwórzmy plik **article_create.html**

```html
{% extends 'base.html' %}

{% block title %}
    Create new article
{% endblock %}

{% block content %}
    <h1>Create article</h1>
    <hr>
    <form method="POST">
        {% csrf_token %}
        Title: <input type="text" name="title">
        <br>
        <textarea name="description" cols="30" rows="10"></textarea>
        <br>
        Author: <select name="author">
            {% for user in users %}
                <option value="{{ user.username }}">{{ user.username }}</option>
            {% endfor %}
        </select>
        <br>
        <button>Create article</button>
    </form>
{% endblock %}
```

Wewnątrz bloku **content** znajduje się formularz z atrybutem **method='POST'**, jako że dane będą wysyłane metodą POST, pamiętaj o tym, jak będziesz chciał przesyłać tajne dane np. hasło żeby była to metoda POST a nie GET, wtedy dane są widoczne w urlu np. `http://mysite.com/login?login=admin&haslo=qwerty`.
Jako, że używamy metody POST, musimy zabezpieczyć nasz formuarz przed atakiem **Cross Site Request Forgery**, zatem musimy użyć tagu **{% csrf_token %}** w przeciwnym wypadku po wysłaniu danych metodą POST zostanie nam zwrócony 403 Forbidden error. Bardzo ważnym atrybutem przy każdym z pól jest atrybut **name** właśnie do tego co wpiszemy wewnątrz niego będziemy się odwoływać w django np:
```html
<input type="password" name="my_pass">
```

```python
# django
password = request.POST["my_pass"]
```

Na końcu jak zawsze zarejestrujmy nasz widok:

```python
from django.contrib import admin
from django.urls import path

from my_app import views 

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", views.home_view, name="home"),
    path("articles/", views.articles, name="articles"),
    path("articles/<int:id>/", views.article_detail, name="article_detail"),
    path("create-article/", views.ArticleCreate.as_view(), name="article_create"),
]
```

Uruchom serwer, i wypróbuj każdy z utworzonych widoków.
Jest to dużo informacji, jednak myślę że zauważyłaś / zauważyłeś schemat: piszemy widok, następnie templatke tego widoku i ostatecznie rejestrujemy ten widok. Jeśli chcesz zobaczyć wszystko w jednym miejscu to przejdź do folderu **Przykladowy-Projekt**.


### Formularze

Kolejną super funkcjonalnością, jaką daje nam Django są formularze. Bardzo często jest tak na stronach, że użytkownik musi się skomunikować z naszą aplikacją przesyłając jej jakieś dane. Tak dzieje się w przypadku logowania, gdy użytkownik wypełnia formularz wpisując login i hasło i wysyła te dane metodą **HTTP POST** jako że nie chcemy wysyłać hasło jawnie metodą GET.







### Zadanie domowe

### Źródła
* [Więcej o CSRF](https://docs.djangoproject.com/en/2.2/ref/csrf/)